In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import pandas as pd

#URL creation for webscrapping 
eg : https://www.indeed.com/jobs?q=python+pandas+Data+Scientist&l=California

In [2]:
#Obtaining an instance of browser
browser = webdriver.Chrome('windows/chromedriver')
#job_type ("Data+Scientist" or "Data+Engineer" or "Data+Analyst"]
#job_state ("Washington" or "New+York" or "Massachusetts" or "Ohio" or "California")

#Variable declarion
job_type = "Data+Scientist"
job_state = "Massachusetts"
skills = "python+pandas+"
job_criteria = skills+job_type
url = "https://www.indeed.com/jobs?q=" + job_criteria + "&l="+job_state
print(url)
browser.get(url)
time.sleep(1)

https://www.indeed.com/jobs?q=python+pandas+Data+Scientist&l=Massachusetts


In [3]:
#Creating a List to hold all the job openings here
job_opening = []
x = 0
scrapping_bool = True
while scrapping_bool:
    x +=1
    # HTML object
    html = browser.page_source
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that have job opening information
    job_listing = soup.find_all('div', class_='jobsearch-SerpJobCard')
    #print(f"this is the no of job listing on webpage {x}")
   # print(f"this is the no of job listing obtained {len(job_title)}")

    for listing in job_listing:
        
        title_detail = listing.find('div',class_='title')
        title = title_detail.a['title']
        
        company_detail = listing.find('div',class_='sjcl')
        company_name = company_detail.span.text.strip()
        
        handle_loc = listing.find('div',class_='recJobLoc')
        location = handle_loc['data-rc-loc'].strip()
        job_summary = listing.find('div',class_='summary').text.strip()
        #Creating a dictionary object for each job opening   
        dict_jobopening = {"job_title": title,"company_name": company_name,"location":location,
                          "job_summary": job_summary}#,"reviews": reviews,"ratings": ratings}
        job_opening.append(dict_jobopening)
        
        time.sleep(1)
       
    try:
        #alert = browser.switch_to_alert()
        #alert.dismiss()
        #To click on the pop-ups that keeps popping up on every page almost
        browser.find_element_by_partial_link_text('No, thanks').click() 
        time.sleep(1)

    except:
        print("No Pop-Up Found.")

    try:
        time.sleep(1)
        print(f"Lenght of list is {len(job_opening)}")
        time.sleep(1)
        print(company_name)
        #Clicking the next link, checking if the word is not a match in the name of the company eg Nextdoor
        nextElem = browser.find_element_by_partial_link_text('Next')
        print(nextElem.text)
        if(nextElem.text == "Next »"):
            nextElem.click()
            print(f"Clicking Next {x}")
            time.sleep(1)
     
    except NoSuchElementException:

        print("Scraping Complete")
        scrapping_bool = False
        break
            
#closing the browser window    
browser.close()


No Pop-Up Found.
Lenght of list is 12
Cambridge Mobile Telematics
Next »
Clicking Next 1
Lenght of list is 23
Qtron Investments
Next »
Clicking Next 2
No Pop-Up Found.
Lenght of list is 34
MassMutual
Next »
Clicking Next 3
No Pop-Up Found.
Lenght of list is 44
Indigo
Next »
Clicking Next 4
No Pop-Up Found.
Lenght of list is 45
DataRobot
Scraping Complete


In [4]:
job_opening_df = pd.DataFrame(job_opening)
job_opening_df.head()




,company_name,job_summary,job_title,location
0,"Iteris, Inc.",Experience or interest in map based data visua...,Software Developer,United States
1,Systems & Technology Research,Your expertise as a data scientist will aid a ...,Data Scientist,"Woburn, MA"
2,Gradient AI,"Synthesize research data, turning data to acti...",Data Scientist,"Cambridge, MA"
3,Facebook,Leverage data and business principles to solve...,"Data Scientist, Infrastructure Strategy Intern","Boston, MA"
4,Harvard University,The data scientist will collaborate with other...,"Data Scientist, Media Manipulation","Cambridge, MA"


In [5]:
# new data frame with split value columns 
new = job_opening_df["location"].str.split(",", n = 1, expand = True) 
#print(new)  
# making separate first name column from new data frame 
job_opening_df["City"]= new[0] 
  
# making separate last name column from new data frame 
job_opening_df["State"]= new[1] 
  
# Dropping old Name columns 
job_opening_df.drop(columns =["location"], inplace = True) 

In [6]:
len(job_opening_df)

45

In [7]:
outFileName = job_state+"_output"+job_type+".xlsx"
print(outFileName)
job_opening_df.to_excel("output/"+outFileName)  
#job_opening_df.to_csv("output/"+outFileName, sep='\t', index=False, header=True, encoding='utf-8')

Massachusetts_outputData+Scientist.xlsx
